# Neural Networks from Scratch - P.4 Batches, Layers, and Objects

# Batch
* What is batch? Send sets of inputs (ensemble) to the neuron instead of one set of input at a time. 
* Why batch? Faster processing. Helps with generalisations (multiple samples at a time).
* **We may overfit if we give all samples at once. We still have to maintain the set structure (batches) while feeding the neuron with input.**

In [31]:
import numpy as np
# batch the input
inputs = np.array([[1, 2, 3, 2.5], 
          [2.0, 5.0, -1.0, 2.0],
          [-1.5, 2.7, 3.3, -0.8]])

# we have 4 input values per node
# 3 nodes in total
# the total number of params that the NN takes is now 4*3 = 12

# weights and biases are associated to the neuron
# so dont have to inc it.
weights = np.array([[0.2, 0.8, -0.5, 1.0], 
           [0.5, -0.91, 0.26, -0.5], 
           [-0.26, -0.27, 0.17, 0.87]] )

bias = np.array([2, 3, 0.5])

## List comprehension and loop over each neuron

In [64]:
output = []

In [65]:
for x in range(0,3): # number of inputs
    output.append([sum([inputs[x][i]*weights[n][i] for i in range(4)])\
             + bias[n] for n in range(n_nodes)])
# %%timeit 
# 90.2 µs ± 1.78 µs per loop
# (mean ± std. dev. of 7 runs, 10000 loops each)

In [66]:
print(output)

[[4.8, 1.21, 2.385], [8.9, -1.8099999999999996, 0.19999999999999996], [1.4100000000000001, 1.0509999999999997, 0.025999999999999912]]


## Numpy and list comprehension at equal exec speeds

In [67]:
print(weights.shape,'and',inputs.shape)

(3, 4) and (3, 4)


We need a 3x3 matrix as result to add with the node bias. So change order, take transpose of weights (matrix product).

In [68]:
%%time 
output_np = np.dot(inputs, weights.T) + bias
# row by row dot product

CPU times: user 98 µs, sys: 5 µs, total: 103 µs
Wall time: 90.1 µs


In [69]:
print(output_np)

[[ 4.8    1.21   2.385]
 [ 8.9   -1.81   0.2  ]
 [ 1.41   1.051  0.026]]


# Add new layer of neurons

Use output of previous layer as input for this one. So we have a 3*3 matrix as input for this layer.

In [70]:
layer1_output = output_np[:] # copy results

In [77]:
# this layer has 3 nodes and each layer gets 3 feature inputs
weights2 = np.array([[0.1, 0.14, 0.5], 
           [0.5, 0.12, -0.33], 
           [-0.44, 0.73, -0.13]])
bias2 = np.array([-1, 2, -0.5]) # each node has its own bias

In [78]:
layer2_output = np.dot(layer1_output,weights2.T) + bias2

In [80]:
print(layer2_output)

[[ 0.8419   3.75815 -2.03875]
 [-0.2634   6.1668  -5.7633 ]
 [-0.69886  2.82254 -0.35655]]


This is a bit untidy, so convert them into objects.

# Object oriented layers

In [119]:
# batch the input
X = np.array([[1, 2, 3, 2.5], 
             [2.0, 5.0, -1.0, 2.0],
             [-1.5, 2.7, 3.3, -0.8]])

Maybe we need to do feature scaling, for faster convergence.

Also need to init the biases (non zero) and weights (ones).

In [120]:
class layer:
    def __init__(self):
        pass
    def forward(self):
        pass

In [121]:
np.random.seed(0) # for reproducibility

In [122]:
class layer:
    def __init__(self, n_inputs, n_neurons):
        self.weights=np.random.randn(n_inputs,n_neurons)
    def forward(self):
        pass

We do it like this to skip the transpose step. `self.weights=np.random.randn(n_inputs,n_neurons)`
(need a weights array (random ndarray) of the size that goes with the input array.) 

But I like doing transposes, so making it the original way.

#### Normalisations
Multiply by 0.1 to make the weights smaller than 1

We also can optimise by doing feature scaling. 
`(xi-mean(x))/std(x)` where `x` is a feature from all the neurons.

In [123]:
class layer:
    def __init__(self, n_inputs, n_neurons):
        self.weights=0.1*np.random.randn(n_neurons,n_inputs)
        self.biases=np.zeros((1,n_neurons)) # 1D bias
    def forward(self, inputs):
        self.output = np.dot(inputs,self.weights.T) + self.biases

In [124]:
layer1 = layer(4,3) # no of features x no of neurons in the layer
layer2 = layer(3,2) # no of features from output of layer 1 x n neurons
layer3 = layer(2,1) # one neuron

No of features from output of layer 1 = no of neurons in previous layer

In [125]:
layer1.forward(X) # forward prop with first layer

In [126]:
print(layer1.output)

[[ 1.11028137  0.23848745  0.47857926]
 [ 0.90319391 -0.24040763  0.46110582]
 [-0.01285333 -0.21836097  0.05753692]]


In [127]:
layer2.forward(layer1.output) # give out of layer 1 to layer 2

In [128]:
print(layer2.output)

[[ 0.10864077  0.0628607 ]
 [ 0.0862781  -0.01524151]
 [-0.00108124 -0.03423416]]


In [129]:
layer3.forward(layer2.output)
print(layer3.output) # final layer took two features and gave one feature output.

[[-0.00196771]
 [ 0.00400286]
 [ 0.00289007]]
